<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Bayesian Linear Mixed Models (Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [1]:
include("../../src/JWAS.jl")

JWAS

In [2]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [3]:
phenofile  = Datasets.dataset("example","phenotypes.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")
genofile   = Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true)
pedigree   = get_pedigree(pedfile,separator=",",header=true);

coding pedigree...  10%|███                             |  ETA: 0:00:01

Finished!


coding pedigree... 100%|████████████████████████████████| Time: 0:00:00


In [4]:
head(phenotypes)

,ID,y1,y2,y3,x1,x2,x3,dam
1,a1,-0.06,3.58,-1.18,0.9,2,m,0
2,a2,-0.6,4.9,0.88,0.3,1,f,0
3,a3,-2.07,3.19,0.73,0.7,2,f,0
4,a4,-2.63,6.97,-0.83,0.6,1,m,a2
5,a5,2.31,3.5,-1.52,0.4,2,m,a2
6,a6,0.93,4.87,-0.01,5.0,2,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Mixed Model (Genomic data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [5]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [6]:
R      = eye(3)
model2 = build_model(model_equation2,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [7]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Build Random or Fixed Effects</button> 

In [8]:
G1 = eye(2)
G2 = eye(4)
set_random(model2,"x2",G1);
set_random(model2,"ID dam",pedigree,G2);

INFO: x2 is not found in model equation 1.
INFO: dam is not found in model equation 2.
INFO: dam is not found in model equation 3.


<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [9]:
G3 = eye(3)
add_genotypes(model2,genofile,G3,separator=',');

5 markers on 10 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [10]:
out2=runMCMC(model2,phenotypes,methods="BayesC",estimatePi=true,chain_length=5000,output_samples_frequency=100);

The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 

 0.462963  0.0       0.0     
 0.0       0.462963  0.0     
 0.0       0.0       0.462963


MCMC Information:

methods                                      BayesC
chain_length                                   5000
burnin                                            0
starting_value                                false
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([1.0, 0.0, 1.0]=>0.0,[0.0, 0.0, 1.0]=>0.0,[0.0, 1.0, 1.0]=>0.0,[1.0, 1.0, 0.0]=>0.0,[0.0, 0.0, 0.0]=>0.0,[0.0, 1.0, 0.0]=>0.0,[1.0, 0.0, 0.0]=>0.0,[1.0, 1.0, 1.0]=>1.0)
estimatePi                           

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:04


In [11]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 7 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Model frequency"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "Posterior mean of location parameters"
  "Posterior mean of Pi"

In [12]:
writedlm("hao.txt",out2["Posterior mean of Pi"])

<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [13]:
using JWAS:misc

In [14]:
get_breeding_values(model2,"MCMC_samples_for_marker_effects_y1.txt",
    "MCMC_samples_for_marker_effects_y2.txt",
    "MCMC_samples_for_marker_effects_y3.txt")

LoadError: [91mSystemError: opening file MCMC_samples_for_marker_effects_y1.txt: No such file or directory[39m

In [15]:
@time for i in 1:500000
issubtype(typeof(rand(2,2)),Number)
end

  0.034993 seconds (500.00 k allocations: 53.406 MiB, 29.54% gc time)


In [16]:
map(String,model2.lhsVec)

3-element Array{String,1}:
 "y1"
 "y2"
 "y3"

In [17]:
  mytraits=map(String,model2.lhsVec)
  thisheader = repeat(mytraits,inner=length(mytraits)).*"_".*repeat(mytraits,outer=length(mytraits))


9-element Array{String,1}:
 "y1_y1"
 "y1_y2"
 "y1_y3"
 "y2_y1"
 "y2_y2"
 "y2_y3"
 "y3_y1"
 "y3_y2"
 "y3_y3"

In [18]:
a=[1.0 2.0 3.0;2.0 4.0 5.0;3.0 5.0 8.0]

3×3 Array{Float64,2}:
 1.0  2.0  3.0
 2.0  4.0  5.0
 3.0  5.0  8.0

In [19]:
vec(a)

9-element Array{Float64,1}:
 1.0
 2.0
 3.0
 2.0
 4.0
 5.0
 3.0
 5.0
 8.0

In [20]:
vec(a')

9-element Array{Float64,1}:
 1.0
 2.0
 3.0
 2.0
 4.0
 5.0
 3.0
 5.0
 8.0